In [1]:

import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

c:\Users\Daniele\Documents\GitHub\CoastSat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_shoreline = gpd.GeoDataFrame.from_features({"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[8.77991,38.913309],[8.780425,38.913113],[8.781026,38.912692],[8.781568,38.912049]]}}]}, crs="EPSG:4326")
new_shoreline

,geometry
0,"LINESTRING (8.77991 38.91331, 8.78042 38.91311..."


In [4]:
new_shoreline.explore(tiles="ESRI.WorldImagery")

In [5]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [7]:
sar_shorelines = shorelines[shorelines.id.str.startswith("sar")]
latest_siteid = sar_shorelines.id.max()
new_siteid = f"sar{int(latest_siteid[3:])+1}"
print(f"Latest siteid is {latest_siteid}, so this new site will be {new_siteid}")
new_shoreline["id"] = new_siteid
new_shoreline

Latest siteid is sar2539, so this new site will be sar2540


,geometry,id
0,"LINESTRING (8.77991 38.91331, 8.78042 38.91311...",sar2540


In [8]:
pd.concat((shorelines, new_shoreline)).to_file("shorelines.geojson", driver="GeoJSON")

In [9]:
new_poly = gpd.GeoDataFrame([{"id": new_siteid}], geometry=new_shoreline.to_crs(32632).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_poly.explore()

In [10]:
pd.concat((poly, new_poly)).to_file("polygons.geojson", driver="GeoJSON")

In [11]:
sar_transects = transects[transects.id.str.startswith("sar")].to_crs(32632)
sar_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
127429,sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.068364,664.0,508.0,0.000113,52.578021,6137.692249,78.343425,124.161754,Medium,"LINESTRING (448682.948 4301601.333, 448711.634..."
127430,sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.197266,664.0,498.0,0.002798,19.969642,2010.059995,44.833693,197.183041,Medium,"LINESTRING (448672.359 4301586.908, 448893.248..."
127431,sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.009191,664.0,545.0,0.000059,11.768691,215.362959,14.675250,206.410997,Medium,"LINESTRING (448718.143 4301576.257, 449018.083..."
127432,sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.040081,664.0,597.0,0.002059,8.475977,112.442187,10.603876,239.306868,Medium,"LINESTRING (448727.641 4301549.522, 449101.288..."
127433,sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.009671,664.0,603.0,0.000141,7.011139,96.998364,9.848775,258.388646,Medium,"LINESTRING (448728.444 4301602.089, 449096.69 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147177,sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.074663,1220.0,1132.0,0.020859,4.031812,37.686052,6.138897,244.745360,Low,"LINESTRING (528106 4571375.787, 527874.719 457..."
147178,sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.159357,388.0,337.0,0.014258,6.685558,118.654858,10.892881,212.777700,Low,"LINESTRING (533505.951 4572159.684, 533629.981..."
147179,sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.160824,388.0,356.0,0.020390,5.452777,82.388602,9.076817,191.174425,Low,"LINESTRING (533416.883 4572348.683, 533780.513..."
147180,sar2539-0002,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,-0.371603,388.0,349.0,0.027053,9.205384,346.372279,18.611079,222.954625,Low,"LINESTRING (533685.132 4572383.49, 533373.453 ..."


In [13]:
def create_transects(line, spacing=50, transect_length=300):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString(reversed([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ]))
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=32632).to_crs(4326)
    return transects

new_transects = create_transects(new_shoreline.to_crs(32632).geometry.iloc[0])
display(new_transects)
m = new_transects.explore()
new_shoreline.explore(m=m)
new_poly.boundary.explore(m=m)
gpd.GeoSeries(new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry
0,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)"
1,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)"
2,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)"
3,"LINESTRING (8.78266 38.91321, 8.77977 38.91172)"
4,"LINESTRING (8.78298 38.91283, 8.78009 38.91134)"


Make sure the origin is inland


In [14]:
new_transects["id"] = new_siteid + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
new_transects["site_id"] = new_siteid
#new_transects["beach_slope"] = .1
new_transects

,geometry,id,site_id
0,"LINESTRING (8.78067 38.91452, 8.77915 38.91209)",sar2540-0000,sar2540
1,"LINESTRING (8.78158 38.91412, 8.77927 38.9121)",sar2540-0001,sar2540
2,"LINESTRING (8.78201 38.91382, 8.7797 38.9118)",sar2540-0002,sar2540
3,"LINESTRING (8.78266 38.91321, 8.77977 38.91172)",sar2540-0003,sar2540
4,"LINESTRING (8.78298 38.91283, 8.78009 38.91134)",sar2540-0004,sar2540


In [15]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")